In [ ]:
import pandas as pd
from datetime import datetime

import yfinance as yf

import talib as ta

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# portfolio of stocks
stocks = ['^GDAXI']

In [ ]:

end_date = datetime.now().strftime("%Y-%m-%d")
start_date = end_date - pd.DateOffset(months=6)

# Download historical data from yf API
data = yf.download(stocks, start=start_date, end=end_date, group_by='ticker')

# # download data to csv
# data.to_csv("stocks_data.csv")